In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
# Import my modules.
import sys, os
from pathlib import Path
current_dir = os.path.join(Path().resolve())
sys.path.append(str(current_dir) + '/../')

from modules import utils
from modules import models
from modules import preprocess

import importlib
for m in [utils, models, preprocess]:
    importlib.reload(m)

In [3]:
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 150)

In [4]:
mbd = 'microbusiness_density'

df_train, df_test, df_subm = utils.load_dataset()
df_all, df_census = utils.merge_dataset(df_train, df_test, pop=False, unemploy=False, census=True, coord=True, fix_pop=True, outlier=True, outlier_method='v2')

/tmp/working/code-analysis/../modules/utils.py:293: DtypeWarning: Columns (102) have mixed types.Specify dtype option on import or set low_memory=False.
  df_census = load_census(BASE)


smooth_outlier: max_scale=v2


TypeError: 'str' object cannot be interpreted as an integer

In [ ]:
params={
        "act_thre": 140,
        "abs_thre": 0,
        "USE_LAG": 7,
        "USE_TREND": True,
        "blacklist": [],
        "blacklistcfips": [],
        "clip": (-0.0044, 0.0046),
        "model": 'lgb',
        "max_window": 12,
        "light": True
    }

## バリデーション

### 前の予測結果を用いて予測していく

In [25]:
accum_pred = models.LgbmBaseline('params_accum',df_subm, df_all, df_census, start_all_dict=35, params=params)
accum_pred.accum_validation(m_len=2, max_pred_m=1)

add lag features: max_scale=35
created df_all_dict[35]
add lag features: max_scale=36
created df_all_dict[36]
add lag features: max_scale=37
created df_all_dict[37]
add lag features: max_scale=38
created df_all_dict[38]
add lag features: max_scale=39
created df_all_dict[39]
add lag features: max_scale=40
created df_all_dict[40]
valid_times:  40
pred_m:  1
train_times:  39
use df_all_dict[39]
['scale', 'state_i', 'pct_bb_2017', 'pct_bb_2018', 'pct_bb_2019', 'pct_bb_2020', 'pct_bb_2021', 'pct_college_2017', 'pct_college_2018', 'pct_college_2019', 'pct_college_2020', 'pct_college_2021', 'pct_foreign_born_2017', 'pct_foreign_born_2018', 'pct_foreign_born_2019', 'pct_foreign_born_2020', 'pct_foreign_born_2021', 'pct_it_workers_2017', 'pct_it_workers_2018', 'pct_it_workers_2019', 'pct_it_workers_2020', 'pct_it_workers_2021', 'median_hh_inc_2017', 'median_hh_inc_2018', 'median_hh_inc_2019', 'median_hh_inc_2020', 'median_hh_inc_2021', 'lng', 'lat', 'select_rate1_lag1', 'select_rate1_lag2', 'se

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


valid_times:  39
pred_m:  1
train_times:  38
use df_all_dict[38]
use lgb.
[LightGBM] [Warning] lambda_l2 is set=0.5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5
[LightGBM] [Warning] min_data_in_leaf is set=213, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=213
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] num_iterations is set=30, n_iter=30 will be ignored. Current value: num_iterations=30
valid_times:  38
pred_m:  1
train_times:  37
use df_all_dict[37]
use lgb.
[LightGBM] [Warning] lambda_l2 is set=0.5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5
[LightGBM] [Warning] min_data_in_leaf is set=213, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=213
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] num_iterations is set=30, n_iter=30 will be ignored. Current value: num_iterations

## Submissionの作成

### 予測結果を活用して予測

In [ ]:
test_subm = models.LgbmBaseline('test_subm_accum', df_subm, df_all, df_census, start_all_dict=40, params=params)
test_subm.create_submission(target_scale=[41, 42, 43, 44], save=False)

In [27]:
df_submission = pd.read_csv('../submission/test_subm_accum_2023-02-25_21:02:46.csv', index_col='row_id')
df_merged = utils.compare_submission(df_submission, 'submission_19346')

In [28]:
df_merged['smape'].mean()

0.009844264000484165

In [29]:
# ensemble
0.01261112205424169

# trend_multi有り
0.03200090765098272

0.03200090765098272

In [52]:
target = 'a' if False else 'b'

In [53]:
target

'b'

In [13]:
dic.get('low_trend_params').get(4).get(params)

AttributeError: 'NoneType' object has no attribute 'get'